# Interactive Recipe & Kitchen Management Assistant

## Step 2: Audio Input & Command Recognition with User Preferences

This notebook implements the second step of our Interactive Recipe & Kitchen Management Assistant capstone project for the Google Gen AI Intensive Course. We'll create a voice interface that allows users to interact with our recipe assistant through spoken commands, recognize different types of user requests, and maintain user preferences.

### Project Overview

The Interactive Recipe & Kitchen Management Assistant helps users:
1. Discover recipes based on available ingredients
2. Customize recipes according to dietary needs
3. Receive step-by-step cooking guidance

This notebook focuses on the **Audio understanding** Gen AI capability, which enables our assistant to:
- Process voice commands using Google Cloud Speech-to-Text
- Interpret user intent from natural language using Gemini Flash model
- Store and retrieve user preferences for personalized experiences

## Setup Environment

Let's set up our environment with the necessary libraries for audio processing, Google Cloud Speech-to-Text, and natural language understanding.

In [10]:
# Setup additional Google API libraries
!pip install -q google-generativeai  # For Gemini API
!pip install -q google-cloud-speech  # For Speech-to-Text
!pip install -q soundfile
!pip install -q pydub  # For audio file handling
!pip install -q ipywidgets

# Install PortAudio dependency for sounddevice
!apt-get update
!apt-get install -y portaudio19-dev #python-pyaudio
!pip install -q sounddevice

!pip install -q spacy
!pip install -q nltk
!pip install -q pandas
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q ipywidgets

# Download necessary NLP models
!python -m spacy download en_core_web_sm
!python -m nltk.downloader punkt
!python -m nltk.downloader stopwords

!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [70.9 kB]                 
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]   
Get:11 https:/

## Import Libraries

Now let's import the libraries we'll need for this step.

In [ ]:
# Import libraries
import os
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import datetime
import random
import warnings
warnings.filterwarnings('ignore')

# Audio processing libraries with error handling
try:
    import soundfile as sf
    import sounddevice as sd
    from IPython.display import Audio, display
    AUDIO_LIBRARIES_AVAILABLE = True
    print("Audio libraries imported successfully!")
except (ImportError, OSError) as e:
    print(f"Warning: Audio libraries could not be imported: {e}")



# Google Cloud Speech-to-Text (with error handling)
try:
    from google.cloud import speech
    GOOGLE_SPEECH_AVAILABLE = True
    print("Google Cloud Speech-to-Text is imported successfully!")
except ImportError:
    GOOGLE_SPEECH_AVAILABLE = False
    print("Google Cloud Speech-to-Text not available. Will use simulation for speech recognition.")

# Google Gemini API for natural language understanding
from google import genai
from google.genai import types
from IPython.display import HTML, Markdown, display
from google.api_core import retry
import IPython.widgets as widgets
from IPython.display import clear_output


# Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
print("Google genai is imported successfully!")


Audio libraries imported successfully!
Google Cloud Speech-to-Text is imported successfully!
Google genai is imported successfully!


### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [13]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

### Run your test prompt

In this step, you will test that your API key is set up correctly by making a request.

The Python SDK uses a [`Client` object](https://googleapis.github.io/python-genai/genai.html#genai.client.Client) to make requests to the API. The client lets you control which back-end to use (between the Gemini API and Vertex AI) and handles authentication (the API key).

The `gemini-2.0-flash` model has been selected here.

**Note**: If you see a `TransportError` on this step, you may need to **🔁 Factory reset** the notebook one time.

In [14]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Hi, This is a test message! How are you?")

print(response.text)

Hi there! I received your test message. I'm doing well, thank you for asking! How can I help you today?



## Load Recipe Data from Step 1

Let's load the recipe data that we processed in Step 1. We'll use this data to test our command recognition system.

In [16]:
# Define paths for loading and saving data
# For Kaggle's output sharing feature
DATA_DIR = Path('/kaggle/input/step1-data-setup')
FINAL_DIR = Path('.')
RECIPE_FILE = FINAL_DIR / 'processed_recipes.json'

# Create data directory if it doesn't exist
DATA_DIR.mkdir(exist_ok=True, parents=True)

# # Try to load the processed recipe data from Step 1
try:
    # Check if the file exists in the Kaggle input directory (if step1 was saved as a dataset)
    kaggle_json_path = DATA_DIR / 'processed_recipes.json'
    
    # First check if the file is in the current directory (where step1 might have saved it)
    if RECIPE_FILE.exists():
        with open(RECIPE_FILE, 'r') as f:
            recipes_data = json.load(f)
        recipes_df = pd.DataFrame(recipes_data)
        print(f"Loaded {len(recipes_df)} recipes from JSON file in current directory")
    
    # Check if JSON file exists in Kaggle input directory
    elif kaggle_json_path.exists():
        with open(kaggle_json_path, 'r') as f:
            recipes_data = json.load(f)
        recipes_df = pd.DataFrame(recipes_data)
        print(f"Loaded {len(recipes_df)} recipes from Kaggle dataset input directory (JSON)")
except Exception as e:
    print(f"\nError loading recipe data: {e}")   
 

Loaded 230186 recipes from Kaggle dataset input directory (JSON)


## Google Cloud Speech-to-Text API Setup

To use Google Cloud Speech-to-Text, we need to set up authentication and configure the client. In a production environment, this would involve creating a service account and downloading the credentials. For demonstration in a Kaggle/local environment, we'll simulate the API response.

> Note: In a real implementation, you would:
> 1. Create a Google Cloud project
> 2. Enable the Speech-to-Text API
> 3. Create a service account with appropriate permissions
> 4. Download the credentials JSON file
> 5. Set the `GOOGLE_APPLICATION_CREDENTIALS` environment variable to point to this file

## Audio Loading and Processing

In a production environment, we would implement real audio recording from the microphone. Since we're in a notebook environment, we'll create functions that load the recorded audios and processing them to demonstrate the workflow.

In [ ]:
# Define audio recording parameters
SAMPLE_RATE = 16000  # 16 kHz
DURATION = 5  # 5 seconds
CHANNELS = 1  # Mono audio


def load_audio_file(file_path, expected_sample_rate=SAMPLE_RATE):
    """
    Load an audio file and convert it to the expected format
    
    Args:
        file_path (str): Path to the audio file
        expected_sample_rate (int): Expected sample rate in Hz
        
    Returns:
        numpy.ndarray: Audio data as numpy array
        int: Sample rate
    """
    try:
        # Check file extension
        file_ext = Path(file_path).suffix.lower()
        
        # For OGG files, use librosa which handles them better
        if file_ext == '.ogg':
            try:
                import librosa
                print(f"Loading OGG file using librosa: {file_path}")
                audio_data, sample_rate = librosa.load(file_path, sr=expected_sample_rate, mono=True)
                
                # Ensure audio_data is a 1D array
                if len(audio_data.shape) > 1:
                    audio_data = audio_data[:, 0]
                
                print(f"Loaded OGG file with sample rate: {sample_rate}Hz")
                return audio_data, sample_rate
            except ImportError:
                print("Librosa not available. Falling back to soundfile.")
        
        # Load the audio file with soundfile
        audio_data, sample_rate = sf.read(file_path)
        
        # Convert to mono if stereo
        if len(audio_data.shape) > 1 and audio_data.shape[1] > 1:
            audio_data = audio_data[:, 0]
        
        # Resample if needed
        if sample_rate != expected_sample_rate:
            # In a real implementation, we would use a proper resampling library
            # For demonstration, we'll just use a simple approach
            print(f"Resampling from {sample_rate}Hz to {expected_sample_rate}Hz")
            audio_data = np.interp(
                np.linspace(0, 1, int(len(audio_data) * expected_sample_rate / sample_rate)),
                np.linspace(0, 1, len(audio_data)),
                audio_data
            )
            sample_rate = expected_sample_rate
        
        return audio_data, sample_rate
    
    except Exception as e:
        print(f"Error loading audio file: {e}")
        return None, None

def preprocess_audio(audio_data, sample_rate=SAMPLE_RATE):
    """
    Preprocess audio data for optimal speech recognition
    
    Args:
        audio_data (numpy.ndarray): Audio data as numpy array
        sample_rate (int): Sample rate in Hz
        
    Returns:
        numpy.ndarray: Preprocessed audio data
    """
    try:
        # Check if input is actually a tuple (audio_data, sample_rate)
        if isinstance(audio_data, tuple) and len(audio_data) == 2:
            print("Warning: You passed a tuple to preprocess_audio. Extracting just the audio data.")
            audio_data, _ = audio_data
        
        # Make sure audio_data is a 1D numpy array
        if not isinstance(audio_data, np.ndarray):
            raise TypeError("Audio data must be a numpy array")
            
        # Ensure it's 1D
        if len(audio_data.shape) > 1:
            print("Converting multi-channel audio to mono")
            audio_data = audio_data.mean(axis=1) if audio_data.shape[1] > 1 else audio_data[:, 0]
        
        # Apply a simple normalization
        if np.max(np.abs(audio_data)) > 0:
            audio_data = audio_data / np.max(np.abs(audio_data)) * 0.9
        
        # Apply a simple noise gate
        noise_threshold = 0.01
        audio_data[np.abs(audio_data) < noise_threshold] = 0
        
        return audio_data
    
    except Exception as e:
        print(f"Error preprocessing audio: {e}")
        return audio_data if isinstance(audio_data, np.ndarray) else np.zeros(1000)  # Return original data or empty array in case of error

## Speech-to-Text Conversion

Let's implement a real speech-to-text function using Google Cloud Speech-to-Text API. This will allow us to convert voice commands from audio files into text for processing.

In [ ]:
def convert_speech_to_text(audio_data, sample_rate=SAMPLE_RATE, language_code="en-US"):
    """
    Convert speech audio to text using Google Cloud Speech-to-Text API
    
    Args:
        audio_data (numpy.ndarray): Audio data as numpy array
        sample_rate (int): Sample rate in Hz
        language_code (str): Language code (e.g., "en-US")
        
    Returns:
        str: Transcribed text
        float: Confidence score (0-1)
    """
    try:
        # Check if we have access to Google Cloud Speech client
        if GOOGLE_SPEECH_AVAILABLE and hasattr(speech, 'SpeechClient'):
            # Initialize the speech client if not already done
            speech_client = speech.SpeechClient()
            
            # Convert the numpy array to bytes
            audio_bytes = (audio_data * 32767).astype(np.int16).tobytes()
            
            # Create recognition config
            config = speech.RecognitionConfig(
                encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
                sample_rate_hertz=sample_rate,
                language_code=language_code,
                enable_automatic_punctuation=True,
                model="default",  # Use "phone_call" for phone audio or "video" for video
                use_enhanced=True  # Use enhanced model
            )
            
            # Create audio object
            audio = speech.RecognitionAudio(content=audio_bytes)
            
            # Send request to the API
            response = speech_client.recognize(config=config, audio=audio)
            
            # Process the response
            if response.results:
                # Get the first alternative (most likely transcription)
                transcription = response.results[0].alternatives[0].transcript
                confidence = response.results[0].alternatives[0].confidence
                
                print(f"Transcribed text: '{transcription}' (confidence: {confidence:.2f})")
                return transcription, confidence
            else:
                print("No speech detected in audio")
                return "", 0.0
        else:
            print("Google Cloud Speech-to-Text API not available.")
            print("Please ensure you have set up the Google Cloud Speech-to-Text API properly.")
            return "", 0.0
        
    except Exception as e:
        print(f"Error in speech-to-text conversion: {e}")
        return "", 0.0

## Command Parsing and Intent Recognition by Gemini Flash

Now let's implement a modern command parsing logic using the Gemini Flash model to extract user intent and entities from transcribed text. This will provide a more accurate and robust understanding of user commands compared to traditional NLP methods.

In [ ]:
# Initialize Gemini model
GEMINI_AVAILABLE = True
try:
    gemini_model = client
    print("Gemini model initialized successfully!")
except NameError:
    GEMINI_AVAILABLE = False
    gemini_model = None
    print("Gemini model not available. Please check your API key.")

def parse_command_with_gemini(text):
    """
    Parse a command to extract intent and entities using Gemini Flash model
    
    Args:
        text (str): The command text
        
    Returns:
        dict: Structured command representation
    """
    try:
        if not GEMINI_AVAILABLE or not gemini_model:
            print("Gemini model not available. Please check your API key.")
            return {
                "text": text,
                "intent": "unknown",
                "confidence": 0.0,
                "ingredients": [],
                "dietary_restrictions": [],
                "cuisine_type": None,
                "meal_type": None,
                "cooking_time": None,
                "timestamp": datetime.datetime.now().isoformat()
            }
        
        # Define the prompt for Gemini to extract intents and entities
        prompt = f"""
        Extract the intent and entities from this cooking command: "{text}"
        
        Classify the intent as one of:
        - find_recipe: Looking for recipes with specific criteria
        - save_preference: Saving user preferences
        - customize_recipe: Customizing or modifying recipes
        - cooking_guidance: Asking for cooking instructions or guidance
        - general_info: Asking for general information
        
        Extract the following entities if present:
        - ingredients: List of food items mentioned
        - dietary_restrictions: Any dietary constraints (vegetarian, gluten-free, etc.)
        - cuisine_type: Type of cuisine (Italian, Mexican, etc.)
        - meal_type: Type of meal (breakfast, dinner, etc.)
        - cooking_time: Time constraints mentioned
        
        Format your response as JSON with the following structure:
        {{
            "intent": "intent_name",
            "confidence": 0.0 to 1.0,
            "entities": {{
                "ingredients": ["item1", "item2"],
                "dietary_restrictions": ["restriction1"],
                "cuisine_type": "cuisine or null",
                "meal_type": "meal type or null",
                "cooking_time": "time description or null"
            }}
        }}
        """
        
        # Generate response from Gemini model
        response = gemini_model.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        
        # Parse the JSON response
        import json
        try:
            result = json.loads(response.text)
            
            # Construct the command structure expected by the rest of the code
            command = {
                "text": text,
                "intent": result["intent"],
                "confidence": float(result["confidence"]),
                "ingredients": result["entities"]["ingredients"] if "ingredients" in result["entities"] else [],
                "dietary_restrictions": result["entities"]["dietary_restrictions"] if "dietary_restrictions" in result["entities"] else [],
                "cuisine_type": result["entities"]["cuisine_type"] if "cuisine_type" in result["entities"] and result["entities"]["cuisine_type"] != "null" else None,
                "meal_type": result["entities"]["meal_type"] if "meal_type" in result["entities"] and result["entities"]["meal_type"] != "null" else None,
                "cooking_time": result["entities"]["cooking_time"] if "cooking_time" in result["entities"] and result["entities"]["cooking_time"] != "null" else None,
                "timestamp": datetime.datetime.now().isoformat()
            }
            
            return command
            
        except json.JSONDecodeError as e:
            print(f"Error parsing Gemini response: {e}")
            print(f"Raw response: {response.text}")
            return {
                "text": text,
                "intent": "unknown",
                "confidence": 0.0,
                "ingredients": [],
                "dietary_restrictions": [],
                "cuisine_type": None,
                "meal_type": None,
                "cooking_time": None,
                "timestamp": datetime.datetime.now().isoformat()
            }
            
    except Exception as e:
        print(f"Error using Gemini for command parsing: {e}")
        return {
            "text": text,
            "intent": "unknown",
            "confidence": 0.0,
            "ingredients": [],
            "dietary_restrictions": [],
            "cuisine_type": None,
            "meal_type": None,
            "cooking_time": None,
            "timestamp": datetime.datetime.now().isoformat()
        }

## Command Confirmation Flow by Gemini Flash

Let's implement a confirmation mechanism using Gemini Flash model to verify we've correctly understood user commands, which is especially important for voice inputs that might be misinterpreted.

In [ ]:
def generate_confirmation_message(command):
    """
    Generate a confirmation message based on the parsed command
    
    Args:
        command (dict): Parsed command structure
        
    Returns:
        str: Confirmation message
    """
    intent = command["intent"]
    message = "I understand you want to "
    
    if intent == "find_recipe":
        message += "find recipes"
        
        # Add ingredients
        if command["ingredients"]:
            message += f" with {', '.join(command['ingredients'])}"
        
        # Add dietary restrictions
        if command["dietary_restrictions"]:
            message += f" that are {', '.join(command['dietary_restrictions'])}"
        
        # Add cuisine type
        if command["cuisine_type"]:
            message += f" in {command['cuisine_type']} cuisine"
        
        # Add meal type
        if command["meal_type"]:
            message += f" for {command['meal_type']}"
        
        # Add cooking time
        if command["cooking_time"]:
            message += f" that are {command['cooking_time']}"
    
    elif intent == "save_preference":
        message += "save your preferences"
        
        # Add dietary restrictions
        if command["dietary_restrictions"]:
            message += f" for {', '.join(command['dietary_restrictions'])} recipes"
        
        # Add cuisine type
        if command["cuisine_type"]:
            message += f" with a preference for {command['cuisine_type']} cuisine"
    
    elif intent == "customize_recipe":
        message += "customize a recipe"
        
        # Add ingredients
        if command["ingredients"]:
            message += f" by replacing or adjusting {', '.join(command['ingredients'])}"
        
        # Add dietary restrictions
        if command["dietary_restrictions"]:
            message += f" to make it {', '.join(command['dietary_restrictions'])}"
    
    elif intent == "cooking_guidance":
        message += "get cooking guidance"
        
        # Add ingredients
        if command["ingredients"]:
            message += f" for cooking with {', '.join(command['ingredients'])}"
    
    elif intent == "general_info":
        message += "get general information"
        
        # Add ingredients
        if command["ingredients"]:
            message += f" about {', '.join(command['ingredients'])}"
    
    else:
        message = f"I'm not sure what you're asking for. Could you rephrase your request?"
    
    message += "."
    return message

def confirm_command(command):
    """
    Simulate a confirmation dialogue with the user
    
    Args:
        command (dict): Parsed command structure
        
    Returns:
        bool: Whether the command was confirmed
        dict: Updated command if modified, original otherwise
    """
    # Generate confirmation message
    confirmation_message = generate_confirmation_message(command)
    print(f"\nConfirmation: {confirmation_message}")
    
    # In a real implementation, we would wait for user confirmation
    # For demonstration, we'll simulate random confirmation/correction
    
    confirmation_result = random.choices(
        ["confirm", "correct", "cancel"],
        weights=[0.7, 0.2, 0.1]
    )[0]
    
    if confirmation_result == "confirm":
        print("User confirmed: Yes, that's correct.")
        return True, command
    
    elif confirmation_result == "correct":
        print("User correction: No, I meant...")
        
        # Simulate a correction
        if command["intent"] == "find_recipe":
            # Add a random ingredient or dietary restriction
            if random.random() > 0.5 and not command["ingredients"]:
                command["ingredients"].append(random.choice(["chicken", "pasta", "vegetables"]))
                print(f"Added ingredient: {command['ingredients'][-1]}")
            elif not command["dietary_restrictions"]:
                command["dietary_restrictions"].append(random.choice(["vegetarian", "gluten-free"]))
                print(f"Added dietary restriction: {command['dietary_restrictions'][-1]}")
        
        # Generate a new confirmation message with the updated command
        updated_confirmation = generate_confirmation_message(command)
        print(f"Updated understanding: {updated_confirmation}")
        print("User: Yes, that's correct now.")
        
        return True, command
    
    else:  # Cancel
        print("User: No, cancel that request.")
        return False, command

## User Preference Storage

Let's implement a system to store and retrieve user preferences. We'll use a simple JSON-based approach for this demonstration.

In [ ]:
# Define path for user preferences
PREFERENCES_FILE = DATA_DIR / 'user_preferences.json'

def load_user_preferences():
    """
    Load user preferences from file
    
    Returns:
        dict: User preferences
    """
    try:
        if PREFERENCES_FILE.exists():
            with open(PREFERENCES_FILE, 'r') as f:
                preferences = json.load(f)
            return preferences
        else:
            # Return default preferences if file doesn't exist
            return {
                "dietary_preferences": [],
                "favorite_recipes": [],
                "avoided_ingredients": [],
                "preferred_cuisines": [],
                "meal_preferences": {},
                "command_history": []
            }
    
    except Exception as e:
        print(f"Error loading user preferences: {e}")
        # Return default preferences in case of error
        return {
            "dietary_preferences": [],
            "favorite_recipes": [],
            "avoided_ingredients": [],
            "preferred_cuisines": [],
            "meal_preferences": {},
            "command_history": []
        }

def save_user_preferences(preferences):
    """
    Save user preferences to file
    
    Args:
        preferences (dict): User preferences to save
        
    Returns:
        bool: Success or failure
    """
    try:
        # Create directory if it doesn't exist
        PREFERENCES_FILE.parent.mkdir(exist_ok=True)
        
        with open(PREFERENCES_FILE, 'w') as f:
            json.dump(preferences, f, indent=2)
        
        print(f"User preferences saved to {PREFERENCES_FILE}")
        return True
    
    except Exception as e:
        print(f"Error saving user preferences: {e}")
        return False

def update_user_preference(preference_type, value):
    """
    Update a specific user preference
    
    Args:
        preference_type (str): Type of preference to update
        value: Value to save
        
    Returns:
        bool: Success or failure
    """
    try:
        # Load current preferences
        preferences = load_user_preferences()
        
        # Update the specific preference
        if preference_type in preferences:
            # For list types, add if not already present
            if isinstance(preferences[preference_type], list):
                if value not in preferences[preference_type]:
                    preferences[preference_type].append(value)
            
            # For dict types, update or add key-value pair
            elif isinstance(preferences[preference_type], dict):
                # Assume value is a dict or tuple/list that can be unpacked
                if isinstance(value, dict):
                    preferences[preference_type].update(value)
                else:
                    key, val = value
                    preferences[preference_type][key] = val
            
            # For other types, simply replace
            else:
                preferences[preference_type] = value
        
        # Save updated preferences
        return save_user_preferences(preferences)
    
    except Exception as e:
        print(f"Error updating user preference: {e}")
        return False

def add_to_command_history(command):
    """
    Add a command to the user's command history
    
    Args:
        command (dict): Command to add to history
        
    Returns:
        bool: Success or failure
    """
    try:
        # Load current preferences
        preferences = load_user_preferences()
        
        # Add the command to history
        if "command_history" in preferences:
            # Limit history to 20 commands
            if len(preferences["command_history"]) >= 20:
                preferences["command_history"].pop(0)
            
            preferences["command_history"].append(command)
        
        # Save updated preferences
        return save_user_preferences(preferences)
    
    except Exception as e:
        print(f"Error adding to command history: {e}")
        return False

def get_user_preference(preference_type=None):
    """
    Get user preferences of a specific type or all preferences
    
    Args:
        preference_type (str, optional): Type of preference to get, or None for all
        
    Returns:
        Any: The preference value(s)
    """
    try:
        # Load preferences
        preferences = load_user_preferences()
        
        # Return specific preference or all preferences
        if preference_type is not None:
            return preferences.get(preference_type, None)
        else:
            return preferences
    
    except Exception as e:
        print(f"Error getting user preference: {e}")
        return None

def process_save_preference_command(command):
    """
    Process a 'save_preference' command and update user preferences
    
    Args:
        command (dict): The parsed command
        
    Returns:
        str: Status message
    """
    try:
        # Check for dietary preferences
        if command["dietary_restrictions"]:
            for preference in command["dietary_restrictions"]:
                update_user_preference("dietary_preferences", preference)
            return f"Saved dietary preferences: {', '.join(command['dietary_restrictions'])}"
        
        # Check for cuisine preferences
        if command["cuisine_type"]:
            update_user_preference("preferred_cuisines", command["cuisine_type"])
            return f"Saved preferred cuisine: {command['cuisine_type']}"
        
        # Check for avoided ingredients
        if command["ingredients"] and ("without" in command["text"].lower() or "avoid" in command["text"].lower()):
            for ingredient in command["ingredients"]:
                update_user_preference("avoided_ingredients", ingredient)
            return f"Saved avoided ingredients: {', '.join(command['ingredients'])}"
        
        # General case for ingredients
        if command["ingredients"]:
            return "Your ingredient preferences have been noted."
        
        return "I'm not sure what preference you want to save. Could you be more specific?"
    
    except Exception as e:
        print(f"Error processing save preference command: {e}")
        return "Sorry, there was an error saving your preferences."

## Text Command Input Alternative

For users who prefer typing over speaking, let's implement a text input interface. In the notebook environment, we'll use IPython widgets to provide an interactive interface.

In [ ]:
def text_command_interface():
    """
    Create an interactive text command interface using IPython widgets
    """
    # Create a text input widget
    text_input = widgets.Text(
        value='',
        placeholder='Type your command (e.g., "Find recipes with chicken and pasta")',
        description='Command:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
    
    # Create an output widget to display results
    output = widgets.Output()
    
    # Define the submit function
    def on_submit(sender):
        with output:
            clear_output()
            process_text_command(text_input.value)
    
    # Connect the submit function to the widget
    text_input.on_submit(on_submit)
    
    # Create a submit button for users who prefer clicking
    submit_button = widgets.Button(
        description='Submit',
        disabled=False,
        button_style='', 
        tooltip='Submit command',
        icon='check'
    )
    
    # Connect the button click to the same function
    submit_button.on_click(lambda b: on_submit(text_input))
    
    # Display the widgets
    display(widgets.HBox([text_input, submit_button]))
    display(output)
    
    print("Type your command and press Enter or click Submit.")

def process_text_command(text):
    """
    Process a text command
    
    Args:
        text (str): The command text
    """
    if not text:
        print("Please enter a command.")
        return
    
    print(f"Processing command: '{text}'")
    
    # Parse the command
    command = parse_command_with_gemini(text)
    
    # Display the parsed command
    print("\nCommand understood as:")
    print(f"Intent: {command['intent']} (confidence: {command['confidence']:.2f})")
    
    if command["ingredients"]:
        print(f"Ingredients: {', '.join(command['ingredients'])}")
    
    if command["dietary_restrictions"]:
        print(f"Dietary restrictions: {', '.join(command['dietary_restrictions'])}")
    
    if command["cuisine_type"]:
        print(f"Cuisine type: {command['cuisine_type']}")
    
    if command["meal_type"]:
        print(f"Meal type: {command['meal_type']}")
    
    if command["cooking_time"]:
        print(f"Cooking time: {command['cooking_time']}")
    
    # Confirm the command
    confirmed, updated_command = confirm_command(command)
    
    if confirmed:
        # Add to command history
        add_to_command_history(updated_command)
        
        # Process according to intent
        if updated_command["intent"] == "find_recipe":
            process_find_recipe_command(updated_command)
        
        elif updated_command["intent"] == "save_preference":
            result = process_save_preference_command(updated_command)
            print(f"\n{result}")
        
        else:
            print(f"\nProcessed {updated_command['intent']} command.")
            print("This functionality will be implemented in a future step.")
    
    else:
        print("\nCommand was cancelled.")

## Unified Voice and Text Interface

Now let's create a unified interface that can handle both voice and text inputs. This simulates what we would implement in a real application.

In [ ]:
def voice_command_interface():
    """
    Create an interactive voice command interface
    """
    # Create a button to start recording
    record_button = widgets.Button(
        description='Start Recording',
        disabled=False,
        button_style='info', 
        tooltip='Start recording voice command',
        icon='microphone'
    )
    
    # Create an output widget to display results
    output = widgets.Output()
    
    # Define the recording function
    def on_record_click(b):
        # Change button appearance during recording
        b.description = 'Recording...'
        b.button_style = 'danger'
        b.icon = 'circle'
        
        with output:
            clear_output()
            
            # Record audio
            audio_data, sample_rate = record_audio(duration=5)
            
            # Preprocess audio
            audio_data = preprocess_audio(audio_data, sample_rate)
            
            # Convert speech to text
            text, confidence = convert_speech_to_text(audio_data, sample_rate)
            
            if text:
                # Process the command text
                process_text_command(text)
            else:
                print("Sorry, I didn't catch that. Please try again.")
        
        # Reset button appearance
        b.description = 'Start Recording'
        b.button_style = 'info'
        b.icon = 'microphone'
    
    # Connect the button click to the recording function
    record_button.on_click(on_record_click)
    
    # Display the widgets
    display(record_button)
    display(output)
    
    print("Click 'Start Recording' and speak your command.")

def process_find_recipe_command(command):
    """
    Process a 'find_recipe' command and display matching recipes
    
    Args:
        command (dict): The parsed command
    """
    print("\nSearching for recipes...")
    
    # Start with all recipes
    filtered_recipes = recipes_df.copy()
    
    # Filter by ingredients if specified
    if command["ingredients"]:
        print(f"Filtering for recipes with: {', '.join(command['ingredients'])}")
        
        # For each specified ingredient, filter recipes that contain it
        for ingredient in command["ingredients"]:
            # Create a pattern to match the ingredient in the ingredients list
            ingredient_pattern = ingredient.lower()
            
            # Filter recipes where any ingredient matches the pattern
            filtered_recipes = filtered_recipes[
                filtered_recipes['ingredients'].apply(
                    lambda ingredients: any(ingredient_pattern in ing.lower() for ing in ingredients)
                    if isinstance(ingredients, list) else False
                )
            ]
    
    # Filter by dietary restrictions if specified
    if command["dietary_restrictions"]:
        print(f"Filtering for {', '.join(command['dietary_restrictions'])} recipes")
        
        # For each specified restriction, filter recipes with that tag
        for restriction in command["dietary_restrictions"]:
            # Create a pattern to match the restriction in the dietary_tags list
            restriction_pattern = restriction.lower()
            
            # Filter recipes where any tag matches the pattern
            filtered_recipes = filtered_recipes[
                filtered_recipes['dietary_tags'].apply(
                    lambda tags: any(restriction_pattern in tag.lower() for tag in tags)
                    if isinstance(tags, list) else False
                )
            ]
    
    # Filter by cuisine type if specified
    if command["cuisine_type"]:
        print(f"Filtering for {command['cuisine_type']} cuisine")
        
        # Filter recipes where cuisine_type matches
        cuisine_pattern = command["cuisine_type"].lower()
        filtered_recipes = filtered_recipes[
            filtered_recipes['cuisine_type'].apply(
                lambda cuisine: cuisine_pattern in cuisine.lower() if cuisine else False
            )
        ]
    
    # Filter by meal type if specified
    if command["meal_type"]:
        print(f"Filtering for {command['meal_type']} recipes")
        
        # For this filter, we would ideally have a 'meal_type' column
        # Since we might not have it in our dataset, we'll check if it exists first
        if 'meal_type' in filtered_recipes.columns:
            meal_pattern = command["meal_type"].lower()
            filtered_recipes = filtered_recipes[
                filtered_recipes['meal_type'].apply(
                    lambda meal: meal_pattern in meal.lower() if meal else False
                )
            ]
        # If no meal_type column, we could try to infer from title or other fields
        else:
            # Look for meal type in recipe title as a simple approach
            meal_pattern = command["meal_type"].lower()
            filtered_recipes = filtered_recipes[
                filtered_recipes['title'].apply(
                    lambda title: meal_pattern in title.lower() if title else False
                )
            ]
    
    # Filter by cooking time if specified
    if command["cooking_time"]:
        print(f"Filtering for recipes that are {command['cooking_time']}")
        
        # Convert cooking time description to numeric filter
        time_desc = command["cooking_time"].lower()
        
        if 'cooking_time' in filtered_recipes.columns:
            if "quick" in time_desc or "fast" in time_desc or "under 30" in time_desc or "less than 30" in time_desc:
                filtered_recipes = filtered_recipes[filtered_recipes['cooking_time'] <= 30]
            elif "hour" in time_desc:
                filtered_recipes = filtered_recipes[filtered_recipes['cooking_time'] >= 60]
    
    # Display results
    if len(filtered_recipes) > 0:
        print(f"\nFound {len(filtered_recipes)} matching recipes:")
        
        # Display the top 5 recipes (or all if less than 5)
        top_recipes = filtered_recipes.head(min(5, len(filtered_recipes)))
        
        for i, (_, recipe) in enumerate(top_recipes.iterrows()):
            print(f"\n{i+1}. {recipe['title']}")
            
            # Display ingredients if available
            if 'ingredients' in recipe and isinstance(recipe['ingredients'], list):
                print(f"   Ingredients: {', '.join(recipe['ingredients'][:5])}" + 
                      (f" and {len(recipe['ingredients']) - 5} more" if len(recipe['ingredients']) > 5 else ""))
            
            # Display cuisine type if available
            if 'cuisine_type' in recipe and recipe['cuisine_type']:
                print(f"   Cuisine: {recipe['cuisine_type']}")
            
            # Display cooking time if available
            if 'cooking_time' in recipe and recipe['cooking_time']:
                print(f"   Cooking Time: {recipe['cooking_time']} minutes")
            
            # Display dietary tags if available
            if 'dietary_tags' in recipe and isinstance(recipe['dietary_tags'], list) and recipe['dietary_tags']):
                print(f"   Dietary Tags: {', '.join(recipe['dietary_tags'])}")
        
        if len(filtered_recipes) > 5:
            print(f"\n... and {len(filtered_recipes) - 5} more recipes.")
    
    else:
        print("\nNo matching recipes found.")
        
        # Provide suggestions for broadening the search
        print("\nTry broadening your search by:")
        if command["ingredients"]:
            print("- Using fewer ingredients")
        if command["dietary_restrictions"]:
            print("- Removing some dietary restrictions")
        if command["cuisine_type"]:
            print("- Trying a different cuisine")

## Complete Workflow: Audio to Action

Let's demonstrate the full workflow from audio input to action execution with a complete example.

In [ ]:
def demonstrate_complete_workflow():
    """
    Demonstrate the complete workflow from audio input to action execution
    """
    print("===== COMPLETE WORKFLOW DEMONSTRATION =====")
    print("\nThis example shows the entire process from audio input to action execution.")
    
    # Step 1: Simulate audio recording
    print("\n1. Recording audio...")
    audio_data, sample_rate = record_audio(duration=3)
    
    # Visualize the audio waveform (simplified for demonstration)
    plt.figure(figsize=(10, 2))
    plt.plot(audio_data)
    plt.title("Audio Waveform")
    plt.xlabel("Sample")
    plt.ylabel("Amplitude")
    plt.tight_layout()
    plt.show()
    
    # Step 2: Preprocess audio
    print("\n2. Preprocessing audio...")
    preprocessed_audio = preprocess_audio(audio_data, sample_rate)
    
    # Step 3: Speech-to-text conversion
    print("\n3. Converting speech to text...")
    # Use a predefined example for demonstration clarity
    text = "Find me a vegetarian recipe with pasta and tomatoes that takes less than 30 minutes"
    confidence = 0.95
    print(f"Transcribed text: '{text}' (confidence: {confidence:.2f})")
    
    # Step 4: Parse command
    print("\n4. Parsing command...")
    command = parse_command_with_gemini(text)
    
    # Print structured command representation
    print("\nStructured command representation:")
    print(json.dumps(command, indent=2))
    
    # Step 5: Confirm command
    print("\n5. Confirming command...")
    confirmation_message = generate_confirmation_message(command)
    print(f"Confirmation: {confirmation_message}")
    print("User: Yes, that's correct.")
    
    # Step 6: Execute command
    print("\n6. Executing command...")
    if command["intent"] == "find_recipe":
        # Search for recipes
        print("\nSearching for recipes with the following criteria:")
        print(f"- Ingredients: {', '.join(command['ingredients'])}")
        print(f"- Dietary restrictions: {', '.join(command['dietary_restrictions'])}")
        print(f"- Cooking time: {command['cooking_time']}")
        
        # Display sample results
        print("\nFound 3 matching recipes:")
        print("1. Quick Vegetarian Pasta Primavera")
        print("   Ingredients: pasta, tomatoes, bell peppers, zucchini, olive oil")
        print("   Cooking Time: 25 minutes")
        print("   Dietary Tags: vegetarian")
        
        print("2. Easy Tomato Basil Penne")
        print("   Ingredients: penne pasta, tomatoes, basil, garlic, olive oil")
        print("   Cooking Time: 20 minutes")
        print("   Dietary Tags: vegetarian, dairy-free")
        
        print("3. 15-Minute Garlic Tomato Spaghetti")
        print("   Ingredients: spaghetti, cherry tomatoes, garlic, olive oil, red pepper flakes")
        print("   Cooking Time: 15 minutes")
        print("   Dietary Tags: vegetarian, dairy-free")
    
    # Step 7: Update user preferences
    print("\n7. Updating user preferences...")
    # Add command to history
    add_to_command_history(command)
    print("Command added to history")
    
    # Update dietary preferences if specified
    if command["dietary_restrictions"]:
        for pref in command["dietary_restrictions"]:
            update_user_preference("dietary_preferences", pref)
        print(f"Updated dietary preferences: {', '.join(command['dietary_restrictions'])}")
    
    print("\nWorkflow demonstration complete!")

## Demonstrate Complete Workflow

Run the cell below to see a demonstration of the complete workflow from audio input to action execution.

In [ ]:
# Demonstrate the complete workflow
demonstrate_complete_workflow()

## Conclusion and Next Steps

In this notebook, we've completed Step 2 of our Interactive Recipe & Kitchen Management Assistant:

1. Implemented audio processing and integration with Google Cloud Speech-to-Text API
2. Created a command parsing system to extract user intent and entities
3. Developed a confirmation flow to verify understood commands
4. Built a user preference storage system that maintains dietary preferences and command history
5. Created a unified interface that supports both voice and text inputs

We've demonstrated the **Audio understanding** Gen AI capability by:
- Converting speech to text using Google Cloud Speech-to-Text
- Parsing natural language commands to extract structured information
- Confirming the system's understanding with the user
- Taking appropriate actions based on understood commands

**Next steps:**
- Step 3: Implement few-shot prompting for recipe customization
- Step 4: Create RAG implementation for recipe knowledge retrieval
- Step 5: Develop function calling capabilities for specific recipe operations

This audio and command recognition system will serve as the foundation for user interaction in our recipe assistant, allowing natural language queries and commands to control the more advanced AI capabilities we'll implement in subsequent steps.

## Notes on Kaggle Environment Adaptation

This notebook has been adapted to work well in the Kaggle environment, which has several limitations for audio processing:

1. **No microphone access**: Kaggle notebooks run in a containerized environment without access to microphone hardware
2. **Limited system library installation**: Installing system dependencies like PortAudio is problematic
3. **Focus on batch processing**: Kaggle is optimized for data science workflows, not real-time audio applications

Our adaptation strategy:
- Use pre-recorded or synthetic audio samples instead of live recording
- Simulate the speech-to-text conversion that would normally use Google Cloud API
- Provide a dropdown to select commands rather than speaking them
- Focus on demonstrating the workflow and Gen AI capabilities, despite the platform limitations

In a production environment running on a system with microphone access and proper API credentials, this code could be easily adapted to use real audio input with minimal changes.